In [ ]:
!nvidia-smi

In [ ]:
!pip install --quiet --upgrade git+https://github.com/huggingface/diffusers.git git+https://github.com/huggingface/transformers.git accelerate

In [ ]:
from diffusers import AudioLDMPipeline
import torch
import scipy
import os


model_id = "cvssp/audioldm-m-full" # model name
pipe = AudioLDMPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

In [ ]:
pipe.to("cuda");
generator = torch.Generator("cuda").manual_seed(0)
negative_prompt = "Low quality, average quality."

In [ ]:
pipe.scheduler.compatibles

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

pipe.enable_model_cpu_offload()

In [ ]:
import pandas as pd

dataset = pd.read_excel('path_excel_file_with_all_the_captions_associated_to_each_file_name_respectively')
descriptions = list(dataset['Selected Caption'])
names = list(dataset['file_name'])
print(descriptions)
print(names)


for idx in range(len(descriptions)):
    audio = pipe(descriptions[idx], negative_prompt=negative_prompt, audio_length_in_s=10, num_inference_steps=20, generator=generator).audios[0]
    scipy.io.wavfile.write(names[idx], rate=16000, data=audio)
    print("saving audio: ", names[idx])
